In [19]:
from dash import Dash, dash_table, dcc, html, Output, Input
import pandas as pd
import plotly.graph_objs as go
import requests

In [20]:
# # you can get the chain list straihgt from the API
# url = 'https://api.llama.fi/v2/chains'
# response = requests.get(url)
# if response.status_code == 200: # data was fetched successfully
#     data = response.json()
#     df = pd.DataFrame(data)
# else:
#     raise Exception('Error fetching data from DeFiLlama API')

# dex_all_chains_url = "https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
# dex_all_chains_response = requests.get(dex_all_chains_url)

# dex_all_chains_data = dex_all_chains_response.json()
# dex_all_chains = set(dex_all_chains_data.get('allChains', []))


# fees_all_chains_url = "https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyFees"
# fees_all_chains_response = requests.get(fees_all_chains_url)

# fees_all_chains_data = fees_all_chains_response.json()
# fees_all_chains = set(fees_all_chains_data.get('allChains', []))

# common_chains = dex_all_chains.intersection(fees_all_chains)

# df = df[['name', 'tvl']]
# df = df[df['name'].isin(common_chains)]

# # Sort the data to get the top 10 chains by TVL
# df = df.sort_values(by='tvl', ascending=False).head(10)

In [21]:
def fetch_historical_data(chain_name, metric):
    if metric == 'tvl':
        url = f'https://api.llama.fi/v2/historicalChainTvl/{chain_name}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)  # Directly create DataFrame from the list of dictionaries
            df['date'] = pd.to_datetime(df['date'], unit='s')
            print(df.tail(120))
            return df[['date', metric]].tail(120)  # Get the last 120 days of the specified metric
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'volume':
        url = (f'https://api.llama.fi/overview/dexs/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyVolume')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            print(df.tail(120))
            return df[['date', metric]].tail(120)
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'fee':
        url = (f'https://api.llama.fi/overview/fees/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyFees')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            print(df.tail(120))
            return df[['date', metric]].tail(120)
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
    else:
        raise ValueError('Invalid metric specified')
    

In [22]:
# Or you can get it thru functions
def fetch_data():
    url = 'https://api.llama.fi/v2/chains'
    response = requests.get(url)
    if response.status_code == 200: # data was fetched successfully
        data = response.json()
        return pd.DataFrame(data)
    else:
        raise Exception('Error fetching data from DeFiLlama API')


# Step 2: Prepare the data
def prepare_data(df):
    url = 'https://api.llama.fi/v2/chains'
    response = requests.get(url)
    if response.status_code == 200: # data was fetched successfully
        data = response.json()
        df = pd.DataFrame(data)
    else:
        raise Exception('Error fetching data from DeFiLlama API')

    dex_all_chains_url = "https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
    dex_all_chains_response = requests.get(dex_all_chains_url)

    dex_all_chains_data = dex_all_chains_response.json()
    dex_all_chains = set(dex_all_chains_data.get('allChains', []))


    fees_all_chains_url = "https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyFees"
    fees_all_chains_response = requests.get(fees_all_chains_url)

    fees_all_chains_data = fees_all_chains_response.json()
    fees_all_chains = set(fees_all_chains_data.get('allChains', []))


    common_chains = dex_all_chains.intersection(fees_all_chains)

    df = df[['name', 'tvl']]
    df = df[df['name'].isin(common_chains)]

    # Sort the data to get the top 10 chains by TVL
    df = df.sort_values(by='tvl', ascending=False).head(10)
    return df


In [ ]:
# Step 3: Create the Dash app and the layout
def create_dashboard(df):
    app = Dash(__name__)
    app.layout = html.Div([
        html.H1('DeFi Chains Dashboard'),
        dcc.RadioItems(
            options=[
                {'label': 'Total Value Locked (TVL)', 'value': 'tvl'},
                {'label': 'Volume', 'value': 'volume'},
                {'label': 'Fees', 'value': 'fee'}
            ],
            id='metric-dropdown',
            value='tvl'
            # value='volume'
            # value='fee'
        ),
        dcc.Graph(id='historical-chart'),
        dash_table.DataTable(
            data=df.to_dict('records'),
            page_size=10,
            style_table={'overflowX': 'auto'}
        )
    ])

    # Step 4: Add callback to update the chart based on selected metric
    @app.callback(
        Output(component_id='historical-chart', component_property='figure'),
        Input(component_id='metric-dropdown', component_property='value')
    )
    def update_charts(selected_metric):
        # Line Chart for historical data of top 10 chains
        historical_fig = go.Figure()
        for chain_name in df['name']:
            historical_data_df = fetch_historical_data(chain_name, selected_metric)
            historical_fig.add_trace(go.Scatter(
                x=historical_data_df['date'],
                y=historical_data_df[selected_metric],
                mode='lines',
                name=chain_name
            ))
        historical_fig.update_layout(title=f'Historical {selected_metric.upper()} of Top 10 Chains (Last 120 Days)')
        
        return historical_fig

    return app

In [24]:
# Step 5: Run the app if the script is executed directly
if __name__ == '__main__':
    df = fetch_data()
    df = prepare_data(df)
    app = create_dashboard(df)
    app.run_server(debug=True, use_reloader=False)  # Disable reloader to prevent multiple fetches of data